In [1]:
import pandas as pd
import numpy as np
from scipy import stats as stats
import regex
import pickle
from matplotlib import pyplot as plt
import requests
from bs4 import BeautifulSoup as bs
from bs4 import Comment
import datetime
from tqdm import tqdm

In [2]:
TEAMS={}
with open('../Teams.txt') as f:
    for r in f.readlines():
        t=r.split(',')
        TEAMS[t[0]]=t[1]

YRs=['2019','2018','2017','2016','2015']
game_details=pd.read_pickle('../Merged DFs/Games_AllData.pkl')

In [3]:
base='https://www.baseball-reference.com/teams/tgl.cgi?team={t}&t=b&year={y}'
gamelogs={}
k=[
    '#','Date',
    'HA','Opp','Outcome',
    'PA','AB',
    'Runs',
    'Hits','2B','3B','HRs',
    'RBI',
    'Walks','Int_Walks',
    'StrikeOuts',
    'Hit_by_Pitch',
    'Sac_Hits','Sac_Flies',
    'Reach_on_Err',
    'DPs_Ground',
    'Stolen_Bases','Caught_Stealing',
    'AVG','OBP','Slugging','OPS','Left_on_Base',
    'Players_Used',
    'Starter_LR','Starter_GScore',
    'link'
    ]

In [4]:
for team in TEAMS.keys():
    print('\r'+team+':  ')
    for yr in YRs:
        l=base.format_map({'t':team,'y':yr})
        r=requests.get(l)
        soup=bs(r.content)
        table=soup.find('table',attrs={'id':'team_batting_gamelogs'}).tbody
        games=table.find_all('tr',attrs={'id':regex.compile(r'team_batting_gamelogs')})
        i=1
        for g in games:
            game_link=g.a['href']
            stats=[d.text for d in g.find_all('td')]
            stats.append(game_link)
            gamelogs[team+'_'+yr+'_'+str(i)]=dict(zip(k,stats))
            i+=1
            print(i,end='\r')

ARI:  
ATL:  
BAL:  
BOS:  
CHC:  
CHW:  
CIN:  
CLE:  
COL:  
DET:  
HOU:  
KCR:  
LAA:  
LAD:  
MIA:  
MIL:  
MIN:  
NYM:  
NYY:  
OAK:  
PHI:  
PIT:  
SDP:  
SFG:  
SEA:  
STL:  
TBR:  
TEX:  
TOR:  
WSN:  


In [5]:
batting_logs=pd.DataFrame.from_dict(gamelogs,orient='index')
batting_logs

,#,Date,HA,Opp,Outcome,PA,AB,Runs,Hits,2B,...,Caught_Stealing,AVG,OBP,Slugging,OPS,Left_on_Base,Players_Used,Starter_LR,Starter_GScore,link
ARI_2019_1,1,Mar 28,@,LAD,"L,5-12",35,33,5,9,4,...,0,.273,.257,.576,.833,3,11,L,H.Ryu(68),/boxes/LAN/LAN201903280.shtml
ARI_2019_2,2,Mar 29,@,LAD,"W,5-4",56,50,5,10,3,...,0,.229,.275,.422,.696,12,18,R,R.Stripling(66),/boxes/LAN/LAN201903290.shtml
ARI_2019_3,3,Mar 30,@,LAD,"L,5-18",35,32,5,7,0,...,0,.226,.278,.470,.747,3,13,R,K.Maeda(56),/boxes/LAN/LAN201903300.shtml
ARI_2019_4,4,Mar 31,@,LAD,"L,7-8",40,39,7,13,2,...,0,.253,.295,.481,.776,6,15,R,W.Buehler(28),/boxes/LAN/LAN201903310.shtml
ARI_2019_5,5,Apr 1,@,SDP,"W,10-3",45,40,10,15,5,...,0,.278,.322,.500,.822,8,11,L,M.Strahm(21),/boxes/SDN/SDN201904010.shtml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSN_2015_158,158,Sep 30,@,ATL,"L,0-2",34,31,0,6,2,...,0,.254,.324,.406,.730,7,11,R,W.Perez(60),/boxes/ATL/ATL201509300.shtml
WSN_2015_159,159,Oct 1,@,ATL,"W,3-0",34,33,3,6,0,...,0,.253,.323,.405,.728,4,15,R,R.Weber(73),/boxes/ATL/ATL201510010.shtml
WSN_2015_160,160,Oct 3 (1),@,NYM,"W,3-1",34,29,3,4,1,...,1,.253,.323,.405,.728,4,15,R,N.Syndergaard(78),/boxes/NYN/NYN201510031.shtml
WSN_2015_161,161,Oct 3 (2),@,NYM,"W,2-0",34,32,2,5,0,...,0,.252,.322,.404,.726,5,10,R,M.Harvey(73),/boxes/NYN/NYN201510032.shtml


In [6]:
batting_logs2=batting_logs.reset_index()
batting_logs2['year']=batting_logs2['index'].str.split('_').apply(lambda l:l[1])
batting_logs2['Team']=batting_logs2['index'].str.split('_').apply(lambda l:l[0])
batting_logs2.Date=batting_logs2.Date.str.extract(r'([A-Z]{1}[a-z]{2} [0-9]{1,2})')
batting_logs2.Date=batting_logs2['Date']+' '+batting_logs2['year']
batting_logs2.Date=batting_logs2.Date.apply(lambda s:pd.to_datetime(s,format='%b %d %Y'))
batting_logs=batting_logs2.set_index(['index'])
batting_logs.Starter_GScore=batting_logs.Starter_GScore.str.extract(r'\((\-?[0-9]{1,3})\)')
batting_logs

,#,Date,HA,Opp,Outcome,PA,AB,Runs,Hits,2B,...,OBP,Slugging,OPS,Left_on_Base,Players_Used,Starter_LR,Starter_GScore,link,year,Team
index,,,,,,,,,,,,,,,,,,,,,
ARI_2019_1,1,2019-03-28,@,LAD,"L,5-12",35,33,5,9,4,...,.257,.576,.833,3,11,L,68,/boxes/LAN/LAN201903280.shtml,2019,ARI
ARI_2019_2,2,2019-03-29,@,LAD,"W,5-4",56,50,5,10,3,...,.275,.422,.696,12,18,R,66,/boxes/LAN/LAN201903290.shtml,2019,ARI
ARI_2019_3,3,2019-03-30,@,LAD,"L,5-18",35,32,5,7,0,...,.278,.470,.747,3,13,R,56,/boxes/LAN/LAN201903300.shtml,2019,ARI
ARI_2019_4,4,2019-03-31,@,LAD,"L,7-8",40,39,7,13,2,...,.295,.481,.776,6,15,R,28,/boxes/LAN/LAN201903310.shtml,2019,ARI
ARI_2019_5,5,2019-04-01,@,SDP,"W,10-3",45,40,10,15,5,...,.322,.500,.822,8,11,L,21,/boxes/SDN/SDN201904010.shtml,2019,ARI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSN_2015_158,158,2015-09-30,@,ATL,"L,0-2",34,31,0,6,2,...,.324,.406,.730,7,11,R,60,/boxes/ATL/ATL201509300.shtml,2015,WSN
WSN_2015_159,159,2015-10-01,@,ATL,"W,3-0",34,33,3,6,0,...,.323,.405,.728,4,15,R,73,/boxes/ATL/ATL201510010.shtml,2015,WSN
WSN_2015_160,160,2015-10-03,@,NYM,"W,3-1",34,29,3,4,1,...,.323,.405,.728,4,15,R,78,/boxes/NYN/NYN201510031.shtml,2015,WSN


In [7]:
batting_logs.to_pickle('../Base DFs/batting_gamelogs.pkl')

In [10]:
patt=r'\/players\/[a-z]{1}\/([a-z\.]+[0-9]{2})\.shtml'
rgx=regex.compile(patt)
find_id=lambda t:rgx.match(t[1]).groups()[0]
A=game_details.Away_Pitcher
H=game_details.Home_Pitcher
#A=A.apply(lambda o:(o[0],o[1],find_id(o)))
#H=H.apply(lambda o:o[1])
pitchers=pd.concat([A,H]).apply(lambda o:(o[0],o[1],find_id(o))).unique()
len(pitchers)

692

In [11]:
base2='https://www.baseball-reference.com/players/gl.fcgi?id={p_id}&t=p&year=20{yr}'
base1='https://www.baseball-reference.com{l}#all_pitching_standard'

In [12]:
pitching_splits={}
pitching_logs={}
for p in tqdm(pitchers):
    name=p[0]
    link=base1.format_map({'l':p[1]})
    player_id=p[2]
    r=requests.get(link)
    soup=bs(r.content)
    try:
        table=soup.find('table',attrs={'id':'pitching_standard'}).tbody
        splits=table.find_all('tr',attrs={'id':regex.compile(r'pitching_standard.20(14|15|16|17|18|19|20)')})
        career=[]
        for yr in splits:
            D={col['data-stat']:col.text for col in yr.find_all('td')}
            D['name']=name
            y=yr['id'].replace('pitching_standard.20','')
            career.append(y)
            pitching_splits[player_id+'_'+y]=D
    except:
        print(name,player_id)
        pass
    for year in career:
        i=0
        try:
            link=base2.format_map({'p_id':player_id,'yr':year})
            r=requests.get(link)
            soup=bs(r.content)
            table=soup.find('table',attrs={'id':'pitching_gamelogs'}).tbody
            games=table.find_all('tr',attrs={'id':regex.compile(r'pitching_gamelogs')})
            for g in games:
                D={col['data-stat']:col.text for col in g.find_all('td')}
                D['year']=int(year)
                D['name']=name
                pitching_logs[player_id+'_'+year+'_'+str(i)]=D
                i+=1
        except:
            pass

 49%|████▊     | 337/692 [14:38<12:33,  2.12s/it]  

Shohei Ohtani ohtansh01


100%|██████████| 692/692 [28:01<00:00,  2.43s/it]


In [13]:
len(pitching_splits)

2709

In [20]:
pitcher_annuals=pd.DataFrame.from_dict(pitching_splits,orient='index')
print(pitcher_annuals.columns)
pitcher_annuals

Index(['age', 'team_ID', 'lg_ID', 'W', 'L', 'win_loss_perc', 'earned_run_avg',
       'G', 'GS', 'GF', 'CG', 'SHO', 'SV', 'IP', 'H', 'R', 'ER', 'HR', 'BB',
       'IBB', 'SO', 'HBP', 'BK', 'WP', 'batters_faced', 'earned_run_avg_plus',
       'fip', 'whip', 'hits_per_nine', 'home_runs_per_nine',
       'bases_on_balls_per_nine', 'strikeouts_per_nine',
       'strikeouts_per_base_on_balls', 'award_summary', 'name'],
      dtype='object')


,age,team_ID,lg_ID,W,L,win_loss_perc,earned_run_avg,G,GS,GF,...,earned_run_avg_plus,fip,whip,hits_per_nine,home_runs_per_nine,bases_on_balls_per_nine,strikeouts_per_nine,strikeouts_per_base_on_balls,award_summary,name
gonzama02_14,22,STL,NL,4,2,.667,4.15,10,5,0,...,88,4.75,1.529,8.3,1.0,5.5,8.0,1.48,,Marco Gonzales
gonzama02_15,23,STL,NL,0,0,,13.50,1,1,0,...,33,8.38,3.000,23.6,3.4,3.4,3.4,1.00,,Marco Gonzales
gonzama02_17,25,TOT,MLB,1,1,.500,6.08,11,8,1,...,70,5.06,1.750,13.3,1.8,2.5,7.2,2.91,,Marco Gonzales
gonzama02_18,26,SEA,AL,13,9,.591,4.00,29,29,0,...,102,3.43,1.224,9.3,0.9,1.7,7.8,4.53,,Marco Gonzales
gonzama02_19,27,SEA,AL,16,13,.552,3.99,34,34,0,...,109,4.15,1.310,9.3,1.0,2.5,6.5,2.63,,Marco Gonzales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
barnema01_19,29,BOS,AL,5,4,.556,3.78,70,0,14,...,129,3.28,1.383,7.1,1.1,5.3,15.4,2.89,,Matt Barnes
barnema01_20,30,BOS,AL,1,3,.250,4.30,24,0,14,...,111,4.84,1.391,7.0,1.6,5.5,12.1,2.21,,Matt Barnes
zychto01_15,24,SEA,AL,0,0,,2.45,13,1,4,...,159,2.04,1.091,8.3,0.5,1.5,11.8,8.00,,Tony Zych
zychto01_16,25,SEA,AL,1,0,1.000,3.29,12,0,3,...,126,2.49,1.463,6.6,0.0,6.6,13.8,2.10,,Tony Zych


In [21]:
pitcher_annuals=pitcher_annuals.rename({'win_loss_perc':'Win_%','earned_run_avg':'ERA','earned_run_avg_plus':'ERA+','hits_per_nine':'H/9','home_runs_per_nine':'HR/9','bases_on_balls_per_nine':'BB/9','strikeouts_per_nine':'K/9','strikeouts_per_base_on_balls':'K/BB'},axis=1)
pitcher_annuals=pitcher_annuals.drop(['award_summary','team_ID','lg_ID'],axis=1)
print(pitcher_annuals.columns)
pitcher_annuals

Index(['age', 'W', 'L', 'Win_%', 'ERA', 'G', 'GS', 'GF', 'CG', 'SHO', 'SV',
       'IP', 'H', 'R', 'ER', 'HR', 'BB', 'IBB', 'SO', 'HBP', 'BK', 'WP',
       'batters_faced', 'ERA+', 'fip', 'whip', 'H/9', 'HR/9', 'BB/9', 'K/9',
       'K/BB', 'name'],
      dtype='object')


,age,W,L,Win_%,ERA,G,GS,GF,CG,SHO,...,batters_faced,ERA+,fip,whip,H/9,HR/9,BB/9,K/9,K/BB,name
gonzama02_14,22,4,2,.667,4.15,10,5,0,0,0,...,156,88,4.75,1.529,8.3,1.0,5.5,8.0,1.48,Marco Gonzales
gonzama02_15,23,0,0,,13.50,1,1,0,0,0,...,16,33,8.38,3.000,23.6,3.4,3.4,3.4,1.00,Marco Gonzales
gonzama02_17,25,1,1,.500,6.08,11,8,1,0,0,...,185,70,5.06,1.750,13.3,1.8,2.5,7.2,2.91,Marco Gonzales
gonzama02_18,26,13,9,.591,4.00,29,29,0,1,0,...,686,102,3.43,1.224,9.3,0.9,1.7,7.8,4.53,Marco Gonzales
gonzama02_19,27,16,13,.552,3.99,34,34,0,0,0,...,866,109,4.15,1.310,9.3,1.0,2.5,6.5,2.63,Marco Gonzales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
barnema01_19,29,5,4,.556,3.78,70,0,14,0,0,...,285,129,3.28,1.383,7.1,1.1,5.3,15.4,2.89,Matt Barnes
barnema01_20,30,1,3,.250,4.30,24,0,14,0,0,...,102,111,4.84,1.391,7.0,1.6,5.5,12.1,2.21,Matt Barnes
zychto01_15,24,0,0,,2.45,13,1,4,0,0,...,76,159,2.04,1.091,8.3,0.5,1.5,11.8,8.00,Tony Zych
zychto01_16,25,1,0,1.000,3.29,12,0,3,0,0,...,60,126,2.49,1.463,6.6,0.0,6.6,13.8,2.10,Tony Zych


In [25]:
pitcher_annuals.to_pickle('../Base DFs/pitcher_years.pkl')

In [26]:
pitcher_gamelogs=pd.DataFrame.from_dict(pitching_logs,orient='index')
print(pitcher_gamelogs.columns)
pitcher_gamelogs

Index(['career_game_num', 'team_game_num', 'date_game', 'team_ID',
       'team_homeORaway', 'opp_ID', 'game_result', 'player_game_span',
       'player_game_result', 'days_rest', 'IP', 'H', 'R', 'ER', 'BB', 'SO',
       'HR', 'HBP', 'earned_run_avg', 'batters_faced', 'pitches',
       'strikes_total', 'strikes_looking', 'strikes_swinging',
       'inplay_gb_total', 'inplay_fb_total', 'inplay_ld', 'inplay_pu',
       'inplay_unk', 'game_score', 'inherited_runners', 'inherited_score',
       'SB', 'CS', 'pickoffs', 'AB', '2B', '3B', 'IBB', 'GIDP', 'SF', 'ROE',
       'leverage_index_avg', 'wpa_def', 'cli_avg', 'cwpa_def', 're24_def',
       'pitcher_situation_in', 'pitcher_situation_out', 'year', 'name',
       'draftkings_points', 'fanduel_points'],
      dtype='object')


,career_game_num,team_game_num,date_game,team_ID,team_homeORaway,opp_ID,game_result,player_game_span,player_game_result,days_rest,...,wpa_def,cli_avg,cwpa_def,re24_def,pitcher_situation_in,pitcher_situation_out,year,name,draftkings_points,fanduel_points
gonzama02_14_0,1,79,Jun 25,STL,@,COL,"W,9-6",GS-5,,99,...,-0.261,1.34,-0.21%,-2.43,1b start tie,5b 3 out d1,14,Marco Gonzales,NaN,NaN
gonzama02_14_1,2,84,Jul 1,STL,@,SFG,"L,0-5",GS-5,L(0-1),5,...,-0.226,1.03,-0.18%,-3.37,1b start tie,5b 123 1 out d4,14,Marco Gonzales,NaN,NaN
gonzama02_14_2,3,89,Jul 6,STL,,MIA,"L,4-8",GS-5,L(0-2),4,...,0.044,1.50,0.04%,0.48,1t start tie,5t 123 2 out d1,14,Marco Gonzales,NaN,NaN
gonzama02_14_3,4,135,Aug 30(2),STL,,CHC,"W,13-2",GS-6,W(1-2),54,...,0.197,2.40,0.25%,1.65,1t start tie,6t 3 out a3,14,Marco Gonzales,NaN,NaN
gonzama02_14_4,5,140,Sep 4,STL,@,MIL,"W,3-2",4-6,W(2-2),4,...,-0.024,2.55,-0.04%,-0.12,4b --- 0 out a2,6b 1-- 1 out a1,14,Marco Gonzales,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zychto01_17_40,66,115,Aug 9,SEA,@,OAK,"W,6-3",8-8,H(11),2,...,0.051,.62,0.03%,0.50,8b --- 0 out a3,8b 3 out a3,17,Tony Zych,NaN,NaN
zychto01_17_41,67,116,Aug 10,SEA,,LAA,"L,3-6",9-GF,,0,...,0.006,.25,0.00%,0.32,9t -2- 2 out d3,9t end d 3,17,Tony Zych,NaN,NaN
zychto01_17_42,68,118,Aug 12,SEA,,LAA,"L,3-6",8-8,L(5-3),1,...,-0.416,1.31,-0.24%,-2.50,8t --- 0 out tie,8t 3 out d3,17,Tony Zych,NaN,NaN
zychto01_17_43,69,122,Aug 16,SEA,,BAL,"W,7-6",5-7,W(6-3),3,...,0.115,1.03,0.04%,1.03,5t 12- 1 out d1,7t 1-- 0 out a3,17,Tony Zych,NaN,NaN


In [27]:
pitcher_gamelogs.player_game_span.str.extract(r'(^[A-Z0-9]{1,2})').value_counts() #takeout non-starts

GS    29903
8      7945
7      7014
9      5268
6      4664
5      2181
4      1198
10      710
3       659
2       430
11      353
CG      242
SH      235
12      179
1       113
13       97
14       48
15       23
16       12
17        7
19        1
dtype: int64

In [28]:
29903+242+235+113 #We want to grab GS, Inning-1, CG and SH

30493

In [29]:
starting_flag=pitcher_gamelogs.player_game_span.str.match(r'^(GS|1-|SH|CG)').values
pitcher_gamelogs=pitcher_gamelogs[starting_flag].copy()
pitcher_gamelogs

,career_game_num,team_game_num,date_game,team_ID,team_homeORaway,opp_ID,game_result,player_game_span,player_game_result,days_rest,...,wpa_def,cli_avg,cwpa_def,re24_def,pitcher_situation_in,pitcher_situation_out,year,name,draftkings_points,fanduel_points
gonzama02_14_0,1,79,Jun 25,STL,@,COL,"W,9-6",GS-5,,99,...,-0.261,1.34,-0.21%,-2.43,1b start tie,5b 3 out d1,14,Marco Gonzales,NaN,NaN
gonzama02_14_1,2,84,Jul 1,STL,@,SFG,"L,0-5",GS-5,L(0-1),5,...,-0.226,1.03,-0.18%,-3.37,1b start tie,5b 123 1 out d4,14,Marco Gonzales,NaN,NaN
gonzama02_14_2,3,89,Jul 6,STL,,MIA,"L,4-8",GS-5,L(0-2),4,...,0.044,1.50,0.04%,0.48,1t start tie,5t 123 2 out d1,14,Marco Gonzales,NaN,NaN
gonzama02_14_3,4,135,Aug 30(2),STL,,CHC,"W,13-2",GS-6,W(1-2),54,...,0.197,2.40,0.25%,1.65,1t start tie,6t 3 out a3,14,Marco Gonzales,NaN,NaN
gonzama02_14_6,7,150,Sep 14,STL,,COL,"W,4-1",GS-6,W(3-2),4,...,0.130,1.26,0.09%,1.25,1t start tie,6t 12- 2 out a2,14,Marco Gonzales,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cingrto01_14_10,37,62,Jun 9,CIN,,LAD,"L,2-6",GS-5,L(2-7),4,...,-0.438,.98,-0.23%,-3.88,1t start tie,5t --- 2 out d4,14,Tony Cingrani,NaN,NaN
cingrto01_15_24,64,93,Jul 22(2),CIN,,CHC,"L,5-6",GS-3,,38,...,-0.323,.28,-0.04%,-4.08,1t start tie,3t 1-3 2 out tie,15,Tony Cingrani,NaN,NaN
barnema01_15_21,27,118,Aug 17,BOS,,CLE,"L,2-8",GS-5,L(3-3),37,...,-0.348,.03,-0.01%,-3.39,1t start tie,5t 3 out d5,15,Matt Barnes,NaN,NaN
barnema01_15_22,28,123,Aug 22,BOS,,KCR,"L,3-6",GS-6,L(3-4),4,...,-0.156,.05,-0.01%,-2.15,1t start tie,6t --- 1 out d5,15,Matt Barnes,NaN,NaN


In [30]:
pitcher_gamelogs.drop(['game_result','player_game_span','player_game_result','team_homeORaway','inherited_runners','inherited_score','wpa_def','cwpa_def','re24_def','pitcher_situation_in','pitcher_situation_out','draftkings_points','fanduel_points'],errors='ignore',axis=1,inplace=True)
#pitcher_gamelogs.date_game=pitcher_gamelogs.date_game.str.replace('(2)','')
print(pitcher_gamelogs.columns)
pitcher_gamelogs

Index(['career_game_num', 'team_game_num', 'date_game', 'team_ID', 'opp_ID',
       'days_rest', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP',
       'earned_run_avg', 'batters_faced', 'pitches', 'strikes_total',
       'strikes_looking', 'strikes_swinging', 'inplay_gb_total',
       'inplay_fb_total', 'inplay_ld', 'inplay_pu', 'inplay_unk', 'game_score',
       'SB', 'CS', 'pickoffs', 'AB', '2B', '3B', 'IBB', 'GIDP', 'SF', 'ROE',
       'leverage_index_avg', 'cli_avg', 'year', 'name'],
      dtype='object')


,career_game_num,team_game_num,date_game,team_ID,opp_ID,days_rest,IP,H,R,ER,...,2B,3B,IBB,GIDP,SF,ROE,leverage_index_avg,cli_avg,year,name
gonzama02_14_0,1,79,Jun 25,STL,COL,99,5.0,7,5,5,...,2,0,0,0,0,0,.99,1.34,14,Marco Gonzales
gonzama02_14_1,2,84,Jul 1,STL,SFG,5,4.1,7,5,5,...,3,0,1,1,0,0,.79,1.03,14,Marco Gonzales
gonzama02_14_2,3,89,Jul 6,STL,MIA,4,4.2,7,1,1,...,0,0,0,1,0,0,1.11,1.50,14,Marco Gonzales
gonzama02_14_3,4,135,Aug 30(2),STL,CHC,54,6.0,3,1,1,...,1,0,0,0,0,1,1.11,2.40,14,Marco Gonzales
gonzama02_14_6,7,150,Sep 14,STL,COL,4,5.2,4,1,1,...,1,0,0,0,0,0,1.10,1.26,14,Marco Gonzales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cingrto01_14_10,37,62,Jun 9,CIN,LAD,4,4.2,7,6,6,...,1,0,0,1,1,0,1.11,.98,14,Tony Cingrani
cingrto01_15_24,64,93,Jul 22(2),CIN,CHC,38,2.2,6,5,5,...,4,0,1,0,0,0,1.32,.28,15,Tony Cingrani
barnema01_15_21,27,118,Aug 17,BOS,CLE,37,5.0,6,6,6,...,2,0,0,0,0,1,.98,.03,15,Matt Barnes
barnema01_15_22,28,123,Aug 22,BOS,KCR,4,5.1,8,5,5,...,1,0,0,1,0,0,.77,.05,15,Matt Barnes


In [31]:
pitcher_gamelogs.year=pitcher_gamelogs.year.apply(lambda y:y+2000)
clean_dates=pitcher_gamelogs.date_game.str.replace(r'\([12]{1}\)','').str.replace('\ssusp','')
clean_dates=clean_dates.replace(u'\xa0',u' ')+u' '+pitcher_gamelogs.year.astype(str)
pitcher_gamelogs.date_game=clean_dates

In [32]:
pitcher_gamelogs['team_yr_game']=pitcher_gamelogs.team_ID+'_'+pitcher_gamelogs.year.astype(str)+'_'+pitcher_gamelogs.team_game_num.astype(str)

In [34]:
pitcher_gamelogs['date']=pitcher_gamelogs.date_game.apply(pd.to_datetime,{'format':'%b %d %Y'})
#pd.to_datetime(,format='%b %d %Y')

In [35]:
print(pitcher_gamelogs.columns)
pitcher_gamelogs

Index(['career_game_num', 'team_game_num', 'date_game', 'team_ID', 'opp_ID',
       'days_rest', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP',
       'earned_run_avg', 'batters_faced', 'pitches', 'strikes_total',
       'strikes_looking', 'strikes_swinging', 'inplay_gb_total',
       'inplay_fb_total', 'inplay_ld', 'inplay_pu', 'inplay_unk', 'game_score',
       'SB', 'CS', 'pickoffs', 'AB', '2B', '3B', 'IBB', 'GIDP', 'SF', 'ROE',
       'leverage_index_avg', 'cli_avg', 'year', 'name', 'team_yr_game',
       'date'],
      dtype='object')


,career_game_num,team_game_num,date_game,team_ID,opp_ID,days_rest,IP,H,R,ER,...,IBB,GIDP,SF,ROE,leverage_index_avg,cli_avg,year,name,team_yr_game,date
gonzama02_14_0,1,79,Jun 25 2014,STL,COL,99,5.0,7,5,5,...,0,0,0,0,.99,1.34,2014,Marco Gonzales,STL_2014_79,2014-06-25
gonzama02_14_1,2,84,Jul 1 2014,STL,SFG,5,4.1,7,5,5,...,1,1,0,0,.79,1.03,2014,Marco Gonzales,STL_2014_84,2014-07-01
gonzama02_14_2,3,89,Jul 6 2014,STL,MIA,4,4.2,7,1,1,...,0,1,0,0,1.11,1.50,2014,Marco Gonzales,STL_2014_89,2014-07-06
gonzama02_14_3,4,135,Aug 30 2014,STL,CHC,54,6.0,3,1,1,...,0,0,0,1,1.11,2.40,2014,Marco Gonzales,STL_2014_135,2014-08-30
gonzama02_14_6,7,150,Sep 14 2014,STL,COL,4,5.2,4,1,1,...,0,0,0,0,1.10,1.26,2014,Marco Gonzales,STL_2014_150,2014-09-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cingrto01_14_10,37,62,Jun 9 2014,CIN,LAD,4,4.2,7,6,6,...,0,1,1,0,1.11,.98,2014,Tony Cingrani,CIN_2014_62,2014-06-09
cingrto01_15_24,64,93,Jul 22 2015,CIN,CHC,38,2.2,6,5,5,...,1,0,0,0,1.32,.28,2015,Tony Cingrani,CIN_2015_93,2015-07-22
barnema01_15_21,27,118,Aug 17 2015,BOS,CLE,37,5.0,6,6,6,...,0,0,0,1,.98,.03,2015,Matt Barnes,BOS_2015_118,2015-08-17
barnema01_15_22,28,123,Aug 22 2015,BOS,KCR,4,5.1,8,5,5,...,0,1,0,0,.77,.05,2015,Matt Barnes,BOS_2015_123,2015-08-22


In [36]:
pitcher_gamelogs.to_pickle('../Base DFs/pitcher_games.pkl')